# NER

In [50]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from huggingface_hub import notebook_login

In [3]:
raw_datasets = load_dataset("conll2003", trust_remote_code=True)
raw_datasets

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
raw_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [10]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [11]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [12]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]

for word, label in zip(words, labels):
    full_label = label_names[label]
    print(f"{word}, label : {full_label}")

EU, label : B-ORG
rejects, label : O
German, label : B-MISC
call, label : O
to, label : O
boycott, label : O
British, label : B-MISC
lamb, label : O
., label : O


### Data Processing

In [14]:
checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

To tokenize a pre-tokenized sentence, we just need to pass is_split_into_words=True

In [18]:
sample_inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(sample_inputs.tokens())

['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


Here, as we can see the word lamb is split into two, creating one extra token. i.e 10 but originally we have only 9 labels for this input, so we need to align the labels with the tokens. For token inside the word, we replace B- with I-. Also for the special tokens, the label will be default to -100

In [19]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    curr_word = None
    for word_id in word_ids:
        if word_id != curr_word:
            curr_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            # Replace B- with I-
            if label%2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

In [21]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = sample_inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [27]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    all_labels = examples["ner_tags"]
    new_labels = []
    
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
        
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [28]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [30]:
tokenized_datasets["train"][0]

{'input_ids': [101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

### FineTuning the model with Trainer API

In [34]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
# Here if we try on a sample of 2 inputs, we can see that it also padded labels to -100 to denote unnecessary labels
sample_batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
sample_batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [36]:
metric = evaluate.load("seqeval")

In [37]:
# To test this metric
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [41]:
predictions = labels.copy()
# introducing an error to test
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [68]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "Overall Precision": all_metrics["overall_precision"],
        "Overall Recall": all_metrics["overall_recall"],
        "Overall F1": all_metrics["overall_f1"],
        "Overall Accuracy": all_metrics["overall_accuracy"]
    }

In [69]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [70]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
model.config.num_labels

9

In [74]:
notebook_login()

In [75]:
args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    processing_class=tokenizer
)
trainer.train()

Epoch,Training Loss,Validation Loss,Overall precision,Overall recall,Overall f1,Overall accuracy
1,0.074800,0.060585,0.909521,0.937227,0.923166,0.982781
2,0.033400,0.065222,0.933245,0.948166,0.940646,0.986019
3,0.019200,0.061365,0.938216,0.950690,0.944412,0.987019


TrainOutput(global_step=5268, training_loss=0.06008250593867552, metrics={'train_runtime': 1028.174, 'train_samples_per_second': 40.969, 'train_steps_per_second': 5.124, 'total_flos': 920771584279074.0, 'train_loss': 0.06008250593867552, 'epoch': 3.0})

In [76]:
trainer.push_to_hub(commit_message="Training complete")

Uploading...:   0%|          | 0.00/431M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/praful-goel/bert-finetuned-ner/commit/1e4e3e8818561c4d8700177e69f1a8daf90b7a72', commit_message='Training complete', commit_description='', oid='1e4e3e8818561c4d8700177e69f1a8daf90b7a72', pr_url=None, repo_url=RepoUrl('https://huggingface.co/praful-goel/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='praful-goel/bert-finetuned-ner'), pr_revision=None, pr_num=None)

### Using the fine-tuned model

In [77]:
from transformers import pipeline

In [78]:
model_checkpoint = "praful-goel/bert-finetuned-ner"
finetuned_pipeline = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")
finetuned_pipeline("My name is Praful Prakash Goel and I study in IIIT Guwahati")

config.json: 0.00B [00:00, ?B/s]

C:\Users\prafu\anaconda3\envs\tf_gpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prafu\.cache\huggingface\hub\models--praful-goel--bert-finetuned-ner. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.99808043,
  'word': 'Praful Prakash Goel',
  'start': 11,
  'end': 30},
 {'entity_group': 'ORG',
  'score': 0.6514068,
  'word': 'IIIT',
  'start': 46,
  'end': 50},
 {'entity_group': 'LOC',
  'score': 0.9334156,
  'word': 'Guwahati',
  'start': 51,
  'end': 59}]